## Feature Engineering
<b> Objective: </b> 
<br>
Add seasonality, promotions

In [12]:
# Feature Engineering
import pandas as pd
import numpy as np
from datetime import datetime

In [13]:
trans_df = pd.read_csv('data/trans.csv')

In [14]:
# --- Seasonality ---
def add_seasonality(transactions_df):
    transactions_df['transaction_date'] = pd.to_datetime(transactions_df['transaction_date'])
    transactions_df['month'] = transactions_df['transaction_date'].dt.month
    transactions_df['seasonal_factor'] = transactions_df['month'].apply(
        lambda m: 1.5 if m in [11,12] else (1.2 if m in [3,4] else 0.8 if m in [1,2] else 1.0)
    )
    transactions_df['adjusted_spend'] = transactions_df['transaction_price'] * transactions_df['seasonal_factor']
    return transactions_df

# --- Promotion Flags ---
def add_promotions(transactions_df, promo_prob=0.15):
    transactions_df['promotion_applied'] = np.random.choice(
        [0,1], size=len(transactions_df), p=[1-promo_prob, promo_prob]
    )
    transactions_df['discount_amount'] = transactions_df.apply(
        lambda row: row['transaction_price']*0.1 if row['promotion_applied']==1 else 0, axis=1
    )
    transactions_df['final_spend'] = transactions_df['transaction_price'] - transactions_df['discount_amount']
    return transactions_df

In [15]:
trans_df_with_seasons = add_seasonality(trans_df)
trans_df_with_promotions = add_promotions(trans_df_with_seasons)

In [16]:
trans_df_with_promotions.columns

Index(['Unnamed: 0', 'transaction_id', 'user_id', 'product_code', 'category',
       'item_name', 'discount_percentage', 'transaction_date',
       'transaction_price', 'age_group', 'gender', 'income_bracket',
       'customer_type', 'state', 'month', 'seasonal_factor', 'adjusted_spend',
       'promotion_applied', 'discount_amount', 'final_spend'],
      dtype='object')

In [17]:
trans_df_with_promotions.to_csv('data/trans_with_promotions.csv')

In [18]:
trans_df_with_promotions [trans_df_with_promotions['promotion_applied'] > 0].head(5)

,Unnamed: 0,transaction_id,user_id,product_code,category,item_name,discount_percentage,transaction_date,transaction_price,age_group,gender,income_bracket,customer_type,state,month,seasonal_factor,adjusted_spend,promotion_applied,discount_amount,final_spend
5,5,dd22f95cce,user_1,3829650,SKIN CARE,Daily Moisturising Light Fragrance Gentle Scen...,0.500000,2023-07-22,26.132987,25-34,Male,50-100K,Occasional,NSW,7,1.0,26.132987,1,2.613299,23.519688
6,6,dd22f95cce,user_1,3762935,SANITARY PROTECTION,The Period Brief Regular Flow Size Medium,0.300505,2023-07-22,20.371381,25-34,Male,50-100K,Occasional,NSW,7,1.0,20.371381,1,2.037138,18.334243
8,8,dd22f95cce,user_1,2777799,BAKING MIXES,Shaved Coconut,0.205479,2023-07-22,3.725937,25-34,Male,50-100K,Occasional,NSW,7,1.0,3.725937,1,0.372594,3.353344
13,13,bc1a7d5fbf,user_1,3628580,PICKLED VEGETABLES,Roasted Red Pepper Strips,0.166667,2023-05-29,6.218921,25-34,Male,50-100K,Occasional,NSW,5,1.0,6.218921,1,0.621892,5.597029
17,17,309c27f96d,user_1,1260335,SNACKS,Honey Soy Chicken Potato Chips,0.500000,2023-09-05,5.248806,25-34,Male,50-100K,Occasional,NSW,9,1.0,5.248806,1,0.524881,4.723925
